In [1]:
# Convert the csv of the translated narratives to an xml tree

from __future__ import unicode_literals, print_function, division
from io import open
import glob
import os
import torch
import sys
from lxml import etree
import argparse
import calendar
import csv
import re
import subprocess
import time
import math 
import time
import math

In [3]:
# -*- coding: UTF-8 -*-
data={}
all_categories = []
tree = etree.parse('../data/mds+rct/train_adult_cat.xml')
for e in tree.iter("cghr_cat"):
        if e.text not in data:
             data[e.text]=[]
             all_categories.append(e.text)
for e in tree.iter("narrative","cghr_cat"):
    if e.tag == "narrative":
        value= u''.join(e.text)
#         print(value)
        
    if e.tag == 'cghr_cat':
        data[e.text].append(value)


# for k,v in data.iteritems():
#     print (k)
#     print ((u"\n".join(v)))

n_categories= len(all_categories)
  
            
            
    
        

In [118]:
all_text = ''
for v in data.itervalues():
    all_text = all_text + u"-".join(v)

vocab = set(all_text)
n_letters = len(vocab)

def letterToIndex(letter):
    return list(vocab).index(letter)

def letterToTensor(letter):
    tensor = torch.zeros(1, n_letters)
    tensor[0][letterToIndex(letter)] = 1
    return tensor

# Turn a line into a <line_length x 1 x n_letters>,
# or an array of one-hot letter vectors
def lineToTensor(narrative):
    tensor = torch.zeros(len(narrative),1)
    for li, letter in enumerate(narrative):
        tensor[li][0] = letterToIndex(letter)
    return tensor

print(letterToTensor('a'))
narr = data['1'][0]
print(lineToTensor(narr))
print(lineToTensor(narr).size())


tensor([[ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.]])
tensor([[   5.],
        [  31.],
        [  64.],
        [  47.],
        [  64.],
        [  68.],
        [   7.],
        [  15.],
        [  64.],
        [  15.],
        [   5.],
        [  65.],
        [ 109.],
        [   5.],
        [  67.],
        [  64.],
        [  15.],
        [ 101.],
  

In [79]:
import torch.nn as nn

class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RNN, self).__init__()

        self.hidden_size = hidden_size

        self.i2h = nn.Linear(input_size + hidden_size, hidden_size)
        self.i2o = nn.Linear(input_size + hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        combined = torch.cat((input, hidden), 1)
        hidden = self.i2h(combined)
        output = self.i2o(combined)
        output = self.softmax(output)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, self.hidden_size)

n_hidden = 128
rnn = RNN(n_letters, n_hidden, n_categories)

In [116]:
class LSTM(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(LSTM, self).__init__()

        self.hidden_size = hidden_size
        self.encoder = nn.Embedding(input_size,20)
        self.lstm = nn.LSTM(20,hidden_size)
        self.linear = nn.Linear(hidden_size,output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        input = self.encoder(input.long())
        output,hidden = self.lstm(input,hidden)
        output = self.linear(output[-1])
        output = self.softmax(output)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, self.hidden_size)

n_hidden = 128
lstm = LSTM(n_letters, n_hidden, n_categories)

In [104]:
input = lineToTensor(narr)
print(input.size())
hidden = torch.zeros(1,1,n_hidden)

output, next_hidden = lstm(input,None)
print(output)
print(output.size())

torch.Size([191, 1])
torch.Size([191, 1, 20])
tensor([[-2.8784, -2.9683, -2.6928, -2.8717, -2.9444, -2.9414, -2.8469,
         -2.9607, -2.9037, -2.8476, -2.8702, -2.9758, -2.9996, -2.8396,
         -2.8535, -2.9868, -2.8491, -2.8452]])
torch.Size([1, 18])


In [122]:
def categoryFromOutput(output):
    top_n, top_i = output.topk(1)
    category_i = top_i[0].item()
    return all_categories[category_i], category_i

print(categoryFromOutput(output))

('19', 2)


In [109]:
import random

def randomChoice(l):
    return l[random.randint(0, len(l) - 1)]

def randomTrainingExample():
    category = randomChoice(all_categories)
    line = randomChoice(data[category])
    category_tensor = torch.tensor([all_categories.index(category)], dtype=torch.long)
    line_tensor = lineToTensor(line)
    return category, line, category_tensor, line_tensor

for i in range(10):
    category, line, category_tensor, line_tensor = randomTrainingExample()
    print('category =', category,category_tensor, '/ line =', line)

category = 1 tensor([ 5]) / line =   को ________ दाी के पास घर मे ही "baby" को "delivery" हुी बचची बिलकुल सवसथ लग रही थी आधे घंटे बाद ही उसने "Bottle" से दूध पीना शुरू कर दिया था और उसी दिन शाम को मां का दूध भी पीना शुरू कर दिया कोी "problem" नही थी चौथे दिन सुबह उसने दूध पिया उसके बाद उसे थोडा सा बुखार लग रहा था घर के पास ही एक डाुक्टर को दिखाया तो उसने बताया की उसे थंड लग गी है दवाी भी दे दी मगर चर पांच घंटे बाद ही उसकी अचानक सांस बंद हो गी और "death" हो गी
category = 19 tensor([ 2]) / line =  बच्चा अस्पताल मे ही जन्म था और जन्म के समय ही कम पैदा हुा था  ही थोदी देर मे खतम हो गया राय मशविरा कर रहे थे कि कया किया जाये कहां ले जाये, उससे पहले वो खतम हो चुका था
category = 12 tensor([ 7]) / line =  उत्तरदाता के अनुसार मृतक कुसुम खातुन की मृत्यु हरदय गति के रुक जाने के कारण हुा कुसुम खातुन को कुछ दिन से कमजोरी था तथा कभी कभी चक्कर भी आता था तथा इसके साथ - साथ कभी कभी कुछ काम करते गिर भी जाती थी उसने इसका इलाज किसी प्रशिकषित डाक्टर से न कराकर गांव के ही ओसा से जाड-फूल करवाने लगी इससे उनकी 

In [110]:
learning_rate = 0.005
criterion = nn.NLLLoss()
optimizer = torch.optim.Adam(lstm.parameters(),lr=learning_rate)

In [113]:
 # If you set this too high, it might explode. If too low, it might not learn

def train(category_tensor, line_tensor):
#     hidden = rnn.initHidden()

    lstm.zero_grad()
    output, hidden = lstm(line_tensor,None)

    loss = criterion(output, category_tensor)
    loss.backward()
    optimizer.step()

    return output, loss.item()

def save():
    save_filename = "./model.pt"
    torch.save(lstm, save_filename)
    print('Saved as %s' % save_filename)
    
def timeSince(since):
    now = time.time()
    s = now - since
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

In [ ]:
if __name__ == '__main__':
    
n_iters = 100000
print_every = 5000
plot_every = 1000

# Keep track of losses for plotting
current_loss = 0
all_losses = []
start = time.time()

for iter in range(1, n_iters + 1):
    category, line, category_tensor, line_tensor = randomTrainingExample()
    output, loss = train(category_tensor, line_tensor)
    current_loss += loss

    # Print iter number, loss, name and guess
    if iter % print_every == 0:
        guess, guess_i = categoryFromOutput(output)
        correct = '✓' if guess == category else '✗ (%s)' % category
        print('%d %d%% (%s) %.4f %s / %s %s' % (iter, iter / n_iters * 100, timeSince(start), loss, line, guess, correct))

    # Add current loss avg to list of losses
    if iter % plot_every == 0:
        all_losses.append(current_loss / plot_every)
        current_loss = 0
save()

In [127]:
evaluator = torch.load("./model.pt")
# -*- coding: UTF-8 -*-
test_data={}

tree = etree.parse('../data/mds+rct/dev_adult_cat.xml')
for e in tree.iter("cghr_cat"):
        if e.text not in test_data:
             test_data[e.text]=[]
for e in tree.iter("narrative","cghr_cat"):
    if e.tag == "narrative":
        value= u''.join(e.text)
#         print(value)
        
    if e.tag == 'cghr_cat':
        test_data[e.text].append(value)

total_test = 0
total_correct = 0
for k,v in test_data.items():
    cat = k
    for line in v:
        narr = line
        narr_tensor = lineToTensor(line)
        output,hidden = evaluator(narr_tensor,None)
        guess, guess_i = categoryFromOutput(output)
        print("correct cat: %s , predicted cat: %s"%(cat,guess))
        total_test +=1
        if cat == guess:
            total_correct +=1

print("total test: %d"%(total_test))
print("total correct: %d"%(total_correct))
print("prediction: %f"%((total_correct/total_test)*100))
            



correct cat: 11 , predicted cat: 3
correct cat: 10 , predicted cat: 12
correct cat: 12 , predicted cat: 2
correct cat: 12 , predicted cat: 2
correct cat: 15 , predicted cat: 8
correct cat: 15 , predicted cat: 17
correct cat: 15 , predicted cat: 19
correct cat: 15 , predicted cat: 19
correct cat: 15 , predicted cat: 9
correct cat: 14 , predicted cat: 14
correct cat: 14 , predicted cat: 13
correct cat: 14 , predicted cat: 2
correct cat: 14 , predicted cat: 3
correct cat: 17 , predicted cat: 9
correct cat: 17 , predicted cat: 9
correct cat: 17 , predicted cat: 14
correct cat: 17 , predicted cat: 9
correct cat: 17 , predicted cat: 18
correct cat: 17 , predicted cat: 4
correct cat: 16 , predicted cat: 3
correct cat: 16 , predicted cat: 18
correct cat: 16 , predicted cat: 15
correct cat: 16 , predicted cat: 8
correct cat: 19 , predicted cat: 17
correct cat: 19 , predicted cat: 10
correct cat: 19 , predicted cat: 3
correct cat: 19 , predicted cat: 19
correct cat: 18 , predicted cat: 2
correct